In [35]:
import pandas as pd
import numpy as np
import os
import utils
from ast import literal_eval

In [36]:
out_filename = 'out.csv'
gts_filename = 'gts.csv'

In [53]:
out_df = pd.read_csv(out_filename)
gts = pd.read_csv(gts_filename)
out_df = out_df.iloc[:1219]
out_df.head()

,Unnamed: 0,x,y,conf,cls_conf,cls_pred,file
0,0,188.800415,314.232239,0.993556,0.981730,1.0,data/custom/images/tr-0001-0000.jpg
1,1,423.422821,214.217880,0.965816,0.988059,1.0,data/custom/images/tr-0001-0001.jpg
2,2,305.167175,302.276917,0.985968,0.952870,1.0,data/custom/images/tr-0002-0000.jpg
3,3,365.646545,353.295441,0.995103,0.994283,1.0,data/custom/images/tr-0002-0001.jpg
4,4,153.367950,389.947449,0.987678,0.986083,1.0,data/custom/images/tr-0002-0002.jpg


In [38]:
gts.head()

,Unnamed: 0,LNDbID,x,y,z,w,h,file,orig shp,origin,t_wrld,lblIndex
0,0,1,0.367188,0.611328,260.0,0.007340,0.007340,tr-0001-0000.jpg,"(328, 512, 512)","(-158.1962890625, -309.1962890625, -297.5)",tr-0001-0000,1
1,1,1,0.591797,0.585938,252.0,0.006472,0.006472,tr-0001-0001.jpg,"(328, 512, 512)","(-158.1962890625, -309.1962890625, -297.5)",tr-0001-0001,1
2,2,2,0.708984,0.691406,170.0,0.008761,0.008761,tr-0002-0000.jpg,"(263, 512, 512)","(-174.1376953125, -380.1376953125, -299.5)",tr-0002-0000,1
3,3,2,0.640625,0.720703,182.0,0.006775,0.006775,tr-0002-0001.jpg,"(263, 512, 512)","(-174.1376953125, -380.1376953125, -299.5)",tr-0002-0001,1
4,4,2,0.189453,0.710938,46.0,0.008090,0.008090,tr-0002-0002.jpg,"(263, 512, 512)","(-174.1376953125, -380.1376953125, -299.5)",tr-0002-0002,1


In [59]:
files = out_df['file']

In [58]:
def fileToId(file_str):
    return int((file_str).split('-')[1].lstrip('0'))

In [57]:
coords = []
for i in range(len(gts)):
    xy = ((out_df[['x', 'y']].iloc[i].values / 512) * literal_eval(gts['orig shp'][i])[1:])
    xyz = np.array([xy[0], xy[1], gts['z'][i]], dtype=int)
    xyz = utils.convertToWorldCoord(xyz,literal_eval(gts['origin'][i]),np.load('.\\TransM\\' + gts['t_wrld'][i] + '.npy'))
    coords.append(xyz)


In [56]:
ids = []
for file in out_df['file'][:1219]:
    ids.append(fileToId(file))

In [60]:
confs = []
for i in range(len(gts)):
    conf = out_df['conf'][i] * out_df['cls_conf'][i] if out_df['cls_pred'][i] == 1 else out_df['conf'][i] * (1 - out_df['cls_conf'][i])
    confs.append(conf)

In [71]:
sub_df = pd.DataFrame(coords, columns=['x','y','z'])
sub_df['LNDbID'] = pd.Series(ids)
sub_df['Nodule'] = pd.Series(confs)

In [73]:
sub_df.head()

,x,y,z,LNDbID,Nodule
0,-44.000977,-118.465820,-37.5,1,0.975403
1,98.743164,-179.208008,-45.5,1,0.954284
2,46.868164,-161.305664,-129.5,2,0.939500
3,90.344727,-124.350586,-117.5,2,0.989414
4,-63.272461,-98.264648,-253.5,2,0.973932


In [77]:
sub_df.to_csv('predictedNodulesA.csv', columns=['LNDbID', 'x', 'y', 'z', 'Nodule'], index=None)